In [2]:
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# 다항식으로 변환한 단항식 생성, [[0,1], [2,3]]의 2x2 행렬 생성
X = np.arange(4).reshape(2,2)
print('일차 단항식 계수 피처:\n', X)

# degree = 2인 2차 다항식으로 변환하기 위해 PolynomialFeatures를 이용해 변환
poly = PolynomialFeatures(degree=2)
poly.fit(X)
poly_ftr = poly.transform(X)
print('변환된 2차 다항식 계수 피처:\n', poly_ftr)

일차 단항식 계수 피처:
 [[0 1]
 [2 3]]
변환된 2차 다항식 계수 피처:
 [[1. 0. 1. 0. 0. 1.]
 [1. 2. 3. 4. 6. 9.]]


- 단항 계수 피처 $[x_1,x_2]$를 2차 다항 계수 $[1, x_1, x_2, x_1^2, x_1x_2, x_2^2]$로 변경하므로 첫 번째 입력 단항 계수 피처 $[x_1 =0, x_2=1]$은 $[1, x_1=0, x_2=1, x_1^2=0, x_1x_2=0, x_2^2=0]$ 형태인 [1, 0, 1, 0, 0, 0, 1]로 변환됨

- 마찬가지로 두 번째 입력 단항 계수 피처 $[x_1=2, x_2=3]$은 [1, 2, 3, 4, 6, 9]로 변환됨
</br>
⇒ 이렇게 변환된 Polynomial 피처에 선형 회귀를 적용해 다항 회귀를 구현

- 이번엔 3차 다항 계수를 이용해 3차 다항 회귀 함수식을 PolynomialFeatures 와 LinearRegression 클래스를 이용해 유도해보자
- 이를 위해 3차 다항 회귀 함수를 임의로 설정하고 이의 회귀 계수를 예측할 것임
- 먼저 3차 다항 회귀의 결정 함수식은 $y=1+2x_1+3x_1^2+4x_2^3$로 설정하고 이를 위한 함수 polynomial_func()을 만듦
    - 해당 함수는 3차 다항 계수 피처 값이 입력되면 결정 값을 반환

In [5]:
def polynomial_func(X):
    # x1 -> X[:,0],  x2 -> X[:,1]
    y = 1 + 2*X[:,0] + 3*X[:,0]**2 + 4*X[:,1]**3 # 3차 다항 회귀의 결정 함수식
    return y

X = np.arange(4).reshape(2,2)
print('일차 단항식 계수 피처:\n', X)
y = polynomial_func(X)
print('3차 다항식 결정값:\n', y)
X[:,0], X[:,1]

일차 단항식 계수 피처:
 [[0 1]
 [2 3]]
3차 다항식 결정값:
 [  5 125]


(array([0, 2]), array([1, 3]))

- 이제 일차 단항식 계수를 삼차 다항식 계수로 변환하고, 이를 선형 회귀에 적용하면 다항 회귀로 구현됨

- PolynomialFeatures(degree=3)은 단항 계수 피처 $[x_1,x_2]$를 3차 다항 계수 $[1, x_1, x_2, x_1^2, x_1x_2, x_2^2, x_1^3, x_1^2x_2, x_1x_2^2, x_2^3 ]$과 같이 10개의 다항 계수로 변환

In [9]:
from sklearn.linear_model import LinearRegression

# 3차 다항식 변환
poly_ftr = PolynomialFeatures(degree=3).fit_transform(X)
print('3차 다항식 계수 feature: \n', poly_ftr)

# Linear Regression에 3차 다항식 계수 faeture와 3차 다항식 결정값으로 학습 후 회귀 계수 확인
model = LinearRegression()
model.fit(poly_ftr, y)
print('Polynomial 회귀 계수\n', np.round(model.coef_, 2))
print('Polynomial 회귀 Shape: ', model.coef_.shape)

3차 다항식 계수 feature: 
 [[ 1.  0.  1.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  2.  3.  4.  6.  9.  8. 12. 18. 27.]]
Polynomial 회귀 계수
 [0.   0.18 0.18 0.36 0.54 0.72 0.72 1.08 1.62 2.34]
Polynomial 회귀 Shape:  (10,)


- 일차 단항식 계수 피처는 2개였지만, 3차 다항식 Polynomial 변환 이후에는 다항식 계수 피처가 10개로 늘어남

- 이 피처 데이터 세트에 LinearRegression을 통해 3차 다항 회귀 형태의 다항 회귀를 적용하면 회귀 계수가 10개로 늘어남

    - 10개의 회귀 계수 [0. 0.18 0.18 0.36 0.54 0.72 0.72 1.08 1.62 2.34]가 도출됐으며</br>
    원래 다항식 $1+2x_1+3x_1^2+4x_2^3$의 계수 값인 [1, 2, 0, 3, 0, 0, 0, 0, 0, 4]와는 차이가 있지만 다항 회귀로 근사하고 있음을 알 수 있음.

피처 변환과 선형 회귀 적용을 각각 별도로 하지않고 사이킷런의 **Pipeline객체를 이용해 한 번에 다항 회귀를 구현 코드**

In [12]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np

def polynomial_func(X):
    y = 1 + 2*X[:,0] + 3*X[:,0]**2 + 4*X[:,1]**3
    return y

# Pipeline 객체로 Streamline하게 Polynomial Feature 변환과 Linear Regression을 연결
model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                 ('linear', LinearRegression())])
X = np.arange(4).reshape(2,2)
y = polynomial_func(X)

model = model.fit(X, y)

print('Polynomial 회귀 계수:\n', np.round(model.named_steps['linear'].coef_, 2))



Polynomial 회귀 계수:
 [0.   0.18 0.18 0.36 0.54 0.72 0.72 1.08 1.62 2.34]
